# Module 2 Peer Review Assignment

**=================================================================================================================**

## Project Description

Building a classification model based on decision tree, and tuning parameters to improve model performance in Rattle 

Use Classification Tree to make a Spam Filter. The data is available in a csv file for you. Its source is Source- DAAG package library R 

## Data Dictionary

| Variable        | Description                                                                           |
|---- ------------|---------------------------------------------------------------------------------------|
| crl.tot|	|
| dollar |	|
| bang |	|
| money |	|
| n000 |	|
| make |	|
| yesno |	|

**=================================================================================================================**

## Import Libraries

In [1]:
import numpy as np
from numpy import count_nonzero, median, mean
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random

# Plotly
# import plotly.express as px
# import plotly.offline as py
# import plotly.graph_objs as go

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

# import researchpy as rp

# import shap
# import eli5
# from IPython.display import display


import datetime
from datetime import datetime, timedelta, date

#import os
#import zipfile

import scipy.stats
from scipy.stats import zscore
from collections import Counter

import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder, PolynomialFeatures

from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score

# from sklearn.feature_selection import f_regression, f_classif, chi2, RFE, RFECV
# from sklearn.feature_selection import mutual_info_regression, mutual_info_classif
# from sklearn.feature_selection import VarianceThreshold, GenericUnivariateSelect
# from sklearn.feature_selection import SelectFromModel, SelectKBest, SelectPercentile

from sklearn.inspection import permutation_importance, plot_partial_dependence

from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error, mean_squared_error,r2_score
from sklearn.metrics import ConfusionMatrixDisplay, PrecisionRecallDisplay, RocCurveDisplay
from sklearn.metrics import auc, f1_score, precision_score, recall_score, roc_auc_score, accuracy_score

from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier, plot_tree

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier


import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

import catboost
from catboost import CatBoostClassifier


%matplotlib inline
#sets the default autosave frequency in seconds
%autosave 60 
sns.set_style('dark')
sns.set(font_scale=1.2)

plt.rc('axes', labelsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

#import imblearn
#from imblearn.under_sampling import RandomUnderSampler
#from imblearn.over_sampling import RandomOverSampler
#from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

# import pickle
# from pickle import dump, load


# PyCaret
from pycaret.classification import *


pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',100)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.2f}'.format)

# Ensure results are reproducible
random.seed(0)
np.random.seed(0)
np.set_printoptions(suppress=True)

Autosaving every 60 seconds


**=================================================================================================================**

## Data Quick Glance

In [2]:
df = pd.read_csv("spam7.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.dtypes.value_counts()

In [ ]:
# Descriptive Statistical Analysis
df.describe(include="all")

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# Check class balance
df['yesno'].value_counts().to_frame()

**=================================================================================================================**

## Data Visualization

In [ ]:
df.hist(bins=50, figsize=(20,25), grid=False, layout=(len(df.columns),2), edgecolor = 'black')
plt.suptitle('Histogram Feature Distribution', x=0.5, y=1.02, ha='center', fontsize=20)
plt.tight_layout()
plt.show()

In [ ]:
df.boxplot(figsize=(20,10), color="blue", fontsize = 15)
plt.title('BoxPlots Feature Distribution', x=0.5, y=1.02, ha='center', fontsize=20)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
sns.countplot(x=df.yesno, data=df)
plt.show()

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(16,9))
sns.heatmap(df.corr(),cmap="coolwarm",annot=True,fmt='.2f',linewidths=2)
plt.title("Correlation Heatmap", fontsize=20)
plt.show()

**==================================================================================================================**

In [3]:
le = LabelEncoder()

In [4]:
df.yesno = le.fit_transform(df.yesno)

In [5]:
# Check class balance
df['yesno'].value_counts().to_frame()

,yesno
0,2788
1,1813


In [6]:
df.head()

,crl.tot,dollar,bang,money,n000,make,yesno
0,278,0.00,0.78,0.00,0.00,0.00,1
1,1028,0.18,0.37,0.43,0.43,0.21,1
2,2259,0.18,0.28,0.06,1.16,0.06,1
3,191,0.00,0.14,0.00,0.00,0.00,1
4,191,0.00,0.14,0.00,0.00,0.00,1


**=================================================================================================================**

## Train Test Split

We've prepared our data and we're ready to model. There's one last step before we can begin. We must split the data into features and target variable, and into training data and test data. We do this using the `train_test_split()` function. We'll put 25% of the data into our test set, and use the remaining 75% to train the model.

Notice below that we include the argument `stratify=y`. If our master data has a class split of 80/20, stratifying ensures that this proportion is maintained in both the training and test data. `=y` tells the function that it should use the class ratio found in the `y` variable (our target).

The less data you have overall, and the greater your class imbalance, the more important it is to stratify when you split the data. If we didn’t stratify, then the function would split the data randomly, and we could get an unlucky split that doesn’t get any of the minority class in the test data, which means we wouldn’t be able to effectively evaluate our model. Worst of all, we might not even realize what went wrong without doing some detective work.

Lastly, we set a random seed so we and others can reproduce our work.

<img src="trin-test.png">

In [ ]:
df.shape

In [ ]:
df.head(1)

In [7]:
X = df.iloc[:,0:6]
y = df.iloc[:,6]

In [8]:
X.values, y.values

(array([[ 278.   ,    0.   ,    0.778,    0.   ,    0.   ,    0.   ],
        [1028.   ,    0.18 ,    0.372,    0.43 ,    0.43 ,    0.21 ],
        [2259.   ,    0.184,    0.276,    0.06 ,    1.16 ,    0.06 ],
        ...,
        [ 118.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.3  ],
        [  78.   ,    0.   ,    0.   ,    0.   ,    0.   ,    0.96 ],
        [  40.   ,    0.   ,    0.125,    0.   ,    0.   ,    0.   ]]),
 array([1, 1, 1, ..., 0, 0, 0]))

In [9]:
train_ratio = 0.70
test_ratio = 0.15
validation_ratio = 0.15

In [10]:
# Splitting the data into train and test subsets (85% for training, 15% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, stratify=y, random_state=0)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3910, 6), (691, 6), (3910,), (691,))

### Train Test Split to create Train, Validation and Test Set

In [12]:
# Splitting the test subset further into test and validation subsets (50% for testing, 50% for validation)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_ratio/(train_ratio+test_ratio), stratify=y_train, random_state=0)

In [13]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((3220, 6), (690, 6), (3220,), (690,))

**=================================================================================================================**

# Feature Scaling

One thing that you can usually try when modeling is scaling your predictor variables. Some models require you to scale the data in order for them to operate as expected, others don't. Naive Bayes does not require data scaling. However, sometimes packages and libraries need to make assumptions and approximations in their calculations. We're already breaking some of these assumptions by using the `GaussianNB` classifier on this dataset, and it may not be helping that some of our predictor variables are on very different scales. In general, scaling might not improve the model, but it probably won't make it worse. Let's try scaling our data.

We'll use a function called `MinMaxScaler`, which we'll import from the `sklearn.preprocessing` module. `MinMaxScaler` normalizes each column so every value falls in the range of [0, 1]. The column's maximum value would scale to 1, and its minimum value would scale to 0. Everything else would fall somewhere between. This is the formula:

$${x_{scaled}} = \frac{x - x_{min}}{x_{max} - x_{min}}$$ 

To use a scaler, you must fit it to the training data, and transform both the training data _and_ the test data using that same scaler. 


One of the most important transformations we need to apply to our data is feature scaling.  There are two common ways to get all attributes to have the same scale: min-max scaling and standardization.

Min-max scaling (or normalization) is the simplest: values are shifted and rescaled so they end up ranging from 0 to 1. This is done by subtracting the min value and dividing by the max minus min.

Standardization is different: first it subtracts the mean value (so standardized values always have a zero mean), and then it divides by the standard deviation, so that the resulting distribution has unit variance.

Scikit-learn library provides `MinMaxScaler` for normalization and `StandardScaler` for standardization needs. For more information on `scikit-learn` [`MinMaxScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0232ENSkillsNetwork837-2023-01-01) and [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0232ENSkillsNetwork837-2023-01-01) please visit their respective documentation websites.

In [14]:
X_train

,crl.tot,dollar,bang,money,n000,make
4191,35,0.00,0.16,0.00,0.00,2.32
1898,56,0.00,0.00,0.00,0.00,0.00
4293,123,0.00,0.00,0.00,0.00,0.00
3016,24,0.00,0.00,0.00,0.00,0.00
1418,288,0.89,0.80,0.00,0.00,0.00
...,...,...,...,...,...,...
1328,339,0.09,1.28,0.00,0.00,0.28
3939,171,0.00,0.00,0.00,0.00,0.00
3839,155,0.04,0.00,0.00,0.00,0.00
1058,1045,0.40,0.98,0.30,0.00,0.10


In [15]:
minmax = MinMaxScaler()

In [16]:
X_train_scaled = minmax.fit_transform(X_train)

In [17]:
X_test_scaled = minmax.transform(X_test)

In [18]:
X_train_scaled

array([[0.00214646, 0.        , 0.0206306 , 0.        , 0.        ,
        0.51101322],
       [0.00347222, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.00770202, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       ...,
       [0.00972222, 0.00749625, 0.        , 0.        , 0.        ,
        0.        ],
       [0.06590909, 0.0661336 , 0.12676787, 0.024     , 0.        ,
        0.02202643],
       [0.0030303 , 0.        , 0.01323472, 0.        , 0.        ,
        0.13876652]])

In [19]:
X_test_scaled

array([[0.00094697, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.00486111, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.20940657, 0.0049975 , 0.0059686 , 0.        , 0.03571429,
        0.        ],
       ...,
       [0.00063131, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.00094697, 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.02973485, 0.        , 0.00233554, 0.        , 0.        ,
        0.        ]])

**=================================================================================================================**

# Model Training

## A reminder about modeling trade-offs

Remember, the decision to use only the champion model to predict on the test data comes with a trade-off. The benefit is that we get a true idea of how we'd expect the model to perform on new, unseen data. The cost of this decision is that, by using the validation scores to both tune hyperparamters _and_ select the champion model, we run the risk of selecting the model that most overfit the validation data. 

Alternatively, we could have selected our champion model by using all of our tuned models to predict on the test data and choosing the one that performed best. That also would have come with a trade-off. There wouldn't be as much risk of overfitting to the validation data, but by using the test data to select our champion model, we wouldn't get a truly objective idea of how the model would perform on new, unseen data. We would need a new dataset for that, which means we would have had to set more data aside at the beginning, resulting in less data to use to train the model. 

With sufficient data, a more rigorous approach would be:

1. Split the data into training, validation, and test sets
2. Tune hyperparameters using cross-validation on the training set
3. Use _all_ tuned models to predict on the validation set
4. Select a champion model based on performance on the validation set
5. Use champion model alone to predict on test data

Every modeling decision comes with a trade-off. What's most important is that you're aware of the trade-offs and apply the best reasoning to the task at hand.


## More rigorous approach:

<img src = "optimal_model_flow.png">

## Decision Tree Model

The `DecisionTreeClassifier` has many arguments (model hyperparameters) that can be customized and eventually tune the generated decision tree classifiers. Among these arguments, there are three commonly tuned arguments as follows:
- criterion: `gini` or `entropy`, which specifies which criteria to be used when splitting a tree node
- max_depth: a numeric value to specify the max depth of the tree. Larger tree depth normally means larger model complexity
- min_samples_leaf: The minimal number of samples in leaf nodes. Larger samples in leaf nodes will tend to generate simpler trees


In [ ]:
dt1 = DecisionTreeClassifier(random_state=0)

In [ ]:
dt1.fit(X_train, y_train)

In [ ]:
dt1pred = dt1.predict(X_test)

In [ ]:
dt1pred[0:5]

In [ ]:
print("Decision Tree Classifier\n")
print('Accuracy:', '%.3f' % accuracy_score(y_test, dt1pred))
print('Precision:', '%.3f' % precision_score(y_test, dt1pred))
print('Recall:', '%.3f' % recall_score(y_test, dt1pred))
print('F1 Score:', '%.3f' % f1_score(y_test, dt1pred))
print('AUC score:', '%3.f' % roc_auc_score(y_test, dt1pred))

### Decision Tree Model Evaluation

To determine which evaluation metric might be best, consider how our model might be wrong. There are two possibilities for bad predictions: 
  
  - **False positives:** When the model predicts a customer **will** churn when in fact they won't
  - **False negatives:** When the model predicts a customer will **not** churn when in fact they will     

As you know, there are a number of performance metrics aside from accuracy to choose from. Some of these include precision, recall, and F1 score. Let's examine these more closely, beginning with _precision_:

$$precision = \frac{\text{TP}}{\text{FP+TP}}$$
  </br> 

And _recall_: 

$$recall = \frac{\text{TP}}{\text{FN+TP}}$$  
  </br>
  
<img src = "confusion matrix.png">

Precision represents the percentage of all our model's predicted positives that are true positives. This might not be the best metric for us to use, because it disincentivizes predicting someone will churn unless there is a high degree of certainty that they will. This could translate to a high rate of false negatives.

On the other hand, recall represents the percentage of all actual positives that the model identifies as such. This also might not be the best metric to use, because it rewards predicting someone will churn even if the likelihood of their doing so is very small. This could translate to a high rate of false positives.

So which is worse, false positives or false negatives? Well, we'd first have to define what _worse_ means. This is dependent on the details of the project that you're working on. For the sake of this exercise, let us suppose that we're defining it as the error that would cost the bank more money.

Since we don't know the exact cost of predicting a false negative, we'll make an assumption for this exercise. We'll assume that a metric that balances precision and recall is best. The metric that helps us achieve this balance is _F1 score_, which is defined as the harmonic mean of precision and recall. 

$${F_{1}} = 2 \cdot \frac{precision \cdot  recall}{precision + recall}$$  
</br>
Again, there are many metrics to choose from. The important thing is that you make an informed decision that is based on your use case.

**Question:** What are the four basic parameters for evaluating the performance of a classification model?

1. True positives (TP): These are correctly predicted positive values, which means the value of actual and predicted classes are positive. 

2. True negatives (TN): These are correctly predicted negative values, which means the value of the actual and predicted classes are negative.

3. False positives (FP): This occurs when the value of the actual class is negative and the value of the predicted class is positive.

4. False negatives (FN): This occurs when the value of the actual class is positive and the value of the predicted class in negative. 

**Reminder:** When fitting and tuning classification modeld, data professioals aim to minimize false positives and false negatives.

**Question:**  What do the four scores demonstrate about your model, and how do you calculate them?

- Accuracy (TP+TN/TP+FP+FN+TN): The ratio of correctly predicted observations to total observations. 
 
- Precision (TP/TP+FP): The ratio of correctly predicted positive observations to total predicted positive observations. 

- Recall (Sensitivity, TP/TP+FN): The ratio of correctly predicted positive observations to all observations in actual class.

- F1 score: The harmonic average of precision and recall, which takes into account both false positives and false negatives. 

In [ ]:
dt1cm = confusion_matrix(y_test, dt1pred)
dt1cm

In [ ]:
print(classification_report(y_test, dt1pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ConfusionMatrixDisplay.from_estimator(estimator=dt1, X=X_test, y=y_test, ax=ax, display_labels=["No","Yes"])
ax.set_title('Confusion matrix of the classifier', size=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

RocCurveDisplay.from_estimator(estimator=dt1, X=X_test, y=y_test, ax=ax)
ax.set_title('ROC Curve of the classifier', size=15)

plt.show()

### Plotting the tree

Next, let's examine the splits of the tree. We'll do this by using the `plot_tree()` function that we imported. We pass to it our fit model as well as some additional parameters. Note that if we did not set `max_depth=2`, the function would return a plot of the entire tree, all the way down to the leaf nodes. This is intractable and unnecessary. We're most interested in the splits nearest the root, because these tell us the most predictive features.

`class_names` displays what the majority class of each node is, and `filled` colors the nodes according to their majority class. 

Note that this plot represents how the tree grew from the _training data_. To make its predictions on the test data, the tree would simply pass each customer in the test data through its splits, from the root node all the way down to a leaf node.

In [ ]:
plt.figure(figsize=(20,12))
plot_tree(dt1, max_depth=2, feature_names=X.columns, class_names=['0','1'], fontsize=14, filled=True)
plt.show()

How do we read this plot? The first line of information in each node is the feature and split point that the model identified as being most predictive. In other words, this is the question that is being asked at that split. For our root node, the question was: _Is the customer less than or equal to 42.5 years old?_

At each node, if the answer to the question it asks is "yes," the sample would move to the child node on the left. If the answer is "no," the sample would go to the child node on the right.

`gini` refers to the node's _Gini impurity_. This is a way of measuring how "pure" a node is. The value can range from 0 to 0.5. A Gini score of 0 means there is no impurity&mdash;the node is a leaf, and all of its samples are of a single class. A score of 0.5 means the classes are all equally represented in that node.

`samples` is simply how many samples are in that node, and `value` indicates how many of each class are in the node. Returning to the root node, we have `value = [5972, 1528]`. Notice that these numbers sum to 7,500, which is the number of samples in the node. This tells us that 5,972 customers in this node stayed (y=0) and 1,528 customers churned (y=1).

Lastly, we have `class`. This tells us the majority class of the samples in each node.

### Build a feature importance graph

Uncover which features might be most important to your decision tree model by building a feature importance graph. 

In [ ]:
importances = dt1.feature_importances_

feature_importances = pd.Series(importances, index=X.columns)

fig, ax = plt.subplots()

feature_importances.plot.bar(ax=ax, figsize=(10,5))
ax.set_title("Decision Tree Feature Importances")
ax.tick_params('x', rotation=30)

fig.show()

In [ ]:
feature_importances_df = pd.DataFrame(feature_importances, columns=["importances"])
feature_importances_df = feature_importances_df.sort_values(by='importances')
feature_importances_df

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.barplot(data=feature_importances_df, x=feature_importances_df.importances, y=feature_importances_df.index, orient='h')

ax.set_title("Decision Tree: Feature Importances for Employee Leaving", fontsize=15)

ax.set_xlabel("Importance")
ax.set_ylabel("Feature")

plt.show()

# PyCaret

In [24]:
# init setup function
s = setup(data=df, target = 'yesno', session_id = 0, normalize=True, normalize_method="minmax",
     fold=5)

,Description,Value
0,Session id,0
1,Target,yesno
2,Target type,Binary
3,Original data shape,"(4601, 7)"
4,Transformed data shape,"(4601, 7)"
5,Transformed train set shape,"(3220, 7)"
6,Transformed test set shape,"(1381, 7)"
7,Numeric features,6
8,Preprocess,True
9,Imputation type,simple


In [25]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8829,0.9239,0.7864,0.9040,0.8411,0.7491,0.7537,1.1480
lightgbm,Light Gradient Boosting Machine,0.8823,0.9182,0.7998,0.8904,0.8426,0.7491,0.7518,0.2640
gbc,Gradient Boosting Classifier,0.8817,0.9230,0.7959,0.8926,0.8412,0.7474,0.7508,0.2740
et,Extra Trees Classifier,0.8755,0.9112,0.8124,0.8636,0.8372,0.7365,0.7375,0.3980
xgboost,Extreme Gradient Boosting,0.8745,0.9169,0.7920,0.8778,0.8326,0.7327,0.7353,0.2740
rf,Random Forest Classifier,0.8720,0.9155,0.8109,0.8571,0.8332,0.7295,0.7305,0.3920
ada,Ada Boost Classifier,0.8655,0.9167,0.7856,0.8613,0.8216,0.7140,0.7162,0.2080
knn,K Neighbors Classifier,0.8559,0.8935,0.7588,0.8596,0.8058,0.6920,0.6956,0.0860
dt,Decision Tree Classifier,0.8398,0.8234,0.7801,0.8073,0.7932,0.6625,0.6630,0.0620
svm,SVM - Linear Kernel,0.8040,0.0000,0.5650,0.9024,0.6940,0.5610,0.5948,0.0480


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [26]:
compare_tree_models = compare_models(include = ['dt', 'rf', 'et', 'gbc'])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.8817,0.9230,0.7959,0.8926,0.8412,0.7474,0.7508,0.1200
et,Extra Trees Classifier,0.8755,0.9112,0.8124,0.8636,0.8372,0.7365,0.7375,0.2080
rf,Random Forest Classifier,0.8720,0.9155,0.8109,0.8571,0.8332,0.7295,0.7305,0.2080
dt,Decision Tree Classifier,0.8398,0.8234,0.7801,0.8073,0.7932,0.6625,0.6630,0.0840


Processing:   0%|          | 0/21 [00:00<?, ?it/s]

In [ ]:
------------

In [ ]:
model_selected = create_model('dt')

In [ ]:
print(model_selected)

In [ ]:
tuned_model = tune_model(catboost, optimize='mse')

In [ ]:
print(tuned_model)

In [ ]:
plot_model(tuned_model)

In [ ]:
plot_model(tuned_model, plot = 'error')

In [ ]:
plot_model(tuned_model, plot='feature')

In [ ]:
interpret_model(tuned_model)

In [ ]:
evaluate_model(tuned_model)

In [ ]:
predict_model(tuned_model)

In [ ]:
final_model = finalize_model(tuned_model)

In [ ]:
unseen_predictions = predict_model(final_model, data=data_unseen)
unseen_predictions.head()

**=================================================================================================================**

## Cross-validated hyperparameter tuning (GridSearchCV)

Cross-validating a model using GridSearchCV can be done in a number of different ways. If you find notebooks online that other people have written, you'll likely soon discover this for yourself. But all variations must fulfill the same general requirements. (Refer to the [GridSearchCV documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) for further reading.)

The format presented below is step-wise, making it easier to follow.

* Create a dictionary of hyperparameters to search over:

  - key = name of hyperparameter (string)
  - value = values to search over (list)
  
* Create a dictionary of scoring metrics to capture. These metrics can be selected from scikit-learn's [built-in options](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter) or custom-defined. For this exercise, we'll capture accuracy, precision, recall, and F1 score so we can examine all of them. The metrics are entered as strings.

* Instantiate the classifier (and set the `random_state`)

* Instantiate the `GridSearchCV` object. Pass as arguments:
  - The classifier (`tuned_decision_tree`)
  - The dictionary of hyperparameters to search over (`tree_para`)
  - The dictionary of scoring metrics (`scoring`)
  - The number of cross-validation folds you want (`cv=5`)
  - The scoring metric that you want GridSearch to use when it selects the "best" model (i.e., the model that performs best on average over all validation folds) (`refit='f1'`*)

    \* The reason it's called `refit` is because once the algorithm finds the combination of hyperparameters that results in the best average score across all validation folds, it will then refit this model to _all_ of the training data. Remember, up until now, with a 5-fold cross-validation, the model has only ever been fit on 80% (4/5) of the training data, because the remaining 20% was held out as a validation fold.

* Fit the data (`X_train`, `y_train`) to the `GridSearchCV` object (`clf`)

Depending on the number of different hyperparameters you choose, the number of combinations you search over, the size of your data, and your available computing resources, this could take a long time. 

Now that the model is fit and cross-validated, we can use the `best_estimator_` attribute to inspect the hyperparameter values that yielded the highest F1 score during cross-validation.

The `best_score_` attribute returns the best average F1 score across the different folds among all the combinations of hyperparameters. Note that if we had set `refit='recall'` when we instantiated our `GridSearchCV` object earlier, then calling `best_score_` would return the best recall score, and the best parameters might not be the same as what they are in the above cell, because the model would be selected based on a different metric.

In [ ]:
# Assign a dictionary of hyperparameters to search over
parameters = {'max_depth': np.arange(2,10,2),
             'min_samples_leaf': np.arange(1,10,2)}

In [ ]:
# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

In [ ]:
# Instantiate the classifier
tuned_decision_tree = DecisionTreeClassifier(random_state = 0)

In [ ]:
# Instantiate the GridSearch
dtgs = GridSearchCV(estimator=tuned_decision_tree, 
                   param_grid=parameters, 
                   scoring = scoring, 
                   cv=5, 
                   refit="roc_auc")

In [ ]:
# Fit the model
%%time
dtgs.fit(X_random_train, y_random_train)

Now that the model is fit and cross-validated, we can use the `best_estimator_` attribute to inspect the hyperparameter values that yielded the highest score during cross-validation.

In [ ]:
# Examine the best model from GridSearch
dtgs.best_estimator_

In [ ]:
dtgs.best_params_

The `best_score_` attribute returns the best average score across the different folds among all the combinations of hyperparameters. Note that if we had set `refit='recall'` when we instantiated our `GridSearchCV` object earlier, then calling `best_score_` would return the best recall score, and the best parameters might not be the same as what they are in the above cell, because the model would be selected based on a different metric.

In [ ]:
dtgs.best_score_

In [ ]:
# we also find the data for all models evaluated

results = pd.DataFrame(dtgs.cv_results_)

print(results.shape)

results.head()

In [ ]:
# we can order the different models based on their performance
results.sort_values(by='mean_test_roc_auc', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[["param_max_depth","param_min_samples_leaf","mean_test_accuracy", "mean_test_recall",
         "mean_test_precision","mean_test_f1","mean_test_roc_auc"]].head()

In [ ]:
def make_results(model_name, model_object):
    '''
    Accepts as arguments a model name (your choice - string) and
    a fit GridSearchCV model object.
  
    Returns a pandas df with the F1, recall, precision, and accuracy scores
    for the model with the best mean F1 score across all validation folds.  
    '''

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(mean f1 score)
    best_estimator_results = cv_results.iloc[cv_results['mean_test_roc_auc'].idxmax(), :]

    # Extract accuracy, precision, recall, and f1 score from that row
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy
    rocauc = best_estimator_results.mean_test_roc_auc
    
    # Create table of results
    table = pd.DataFrame()
    table = table.append({'Model': model_name,
                        'F1': f1,
                        'Recall': recall,
                        'Precision': precision,
                        'Accuracy': accuracy,
                        'ROC-AUC' : rocauc
                        },
                        ignore_index=True
                       )
  
    return table

In [ ]:
make_results(model_name="Decision Tree GSCV", model_object=dtgs)

### Using RandomSearchCV

Randomised grid search is very useful in finding near-optimal hyper parameters for any machine learning models.

Rules of thumb: with 60 iterations, 95% of the time, best 5% sets of parameters can be found, regardless of grid size.

In [ ]:
dt = DecisionTreeClassifier(random_state=0)

In [ ]:
parameters = { 'criterion': ['gini', 'entropy', 'log_loss'],
               'max_depth': np.arange(2,11,2),
               'min_samples_split': np.arange(2,21,3),
               'min_samples_leaf':np.arange(1,11,2)
             }

In [ ]:
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

In [ ]:
dt_randm = RandomizedSearchCV(estimator=dt, param_distributions = parameters, cv = 5, n_iter = 55, 
                           n_jobs=2, scoring=scoring, refit='roc_auc', return_train_score=True)

In [ ]:
%%time
dt_randm.fit(X_random_train, y_random_train)

In [ ]:
dt_randm.best_estimator_

In [ ]:
dt_randm.best_score_

In [ ]:
dt_randm.best_params_

In [ ]:
# we also find the data for all models evaluated

results = pd.DataFrame(dt_randm.cv_results_)

print(results.shape)

results.head()

In [ ]:
# we can order the different models based on their performance
results.sort_values(by='mean_test_roc_auc', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[["param_min_samples_split","param_max_depth","param_min_samples_leaf","mean_test_accuracy", "mean_test_recall",
         "mean_test_precision","mean_test_f1","mean_test_roc_auc"]].head()

In [ ]:
def make_results(model_name, model_object):
    '''
    Accepts as arguments a model name (your choice - string) and
    a fit GridSearchCV model object.
  
    Returns a pandas df with the F1, recall, precision, and accuracy scores
    for the model with the best mean F1 score across all validation folds.  
    '''

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(mean f1 score)
    best_estimator_results = cv_results.iloc[cv_results['mean_test_roc_auc'].idxmax(), :]

    # Extract accuracy, precision, recall, and f1 score from that row
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy
    rocauc = best_estimator_results.mean_test_roc_auc
  
    # Create table of results
    table = pd.DataFrame()
    table = table.append({'Model': model_name,
                        'F1': f1,
                        'Recall': recall,
                        'Precision': precision,
                        'Accuracy': accuracy,
                        'ROC-AUC' : rocauc  
                        },
                        ignore_index=True
                       )
  
    return table

In [ ]:
# Call the function on our model
dtrs_result_table = make_results("Decision Tree RCV", dt_randm)

In [ ]:
dtrs_result_table

**=================================================================================================================**

### Tuned Decision Tree Model

In [ ]:
dt2 = DecisionTreeClassifier(criterion='gini', min_samples_split=11, min_samples_leaf=3, max_depth=8)

In [ ]:
dt2.fit(X_train, y_train)

In [ ]:
dt2pred = dt2.predict(X_test)

In [ ]:
dt2pred[0:5]

In [ ]:
print('Accuracy:', '%.3f' % accuracy_score(y_test, dt2pred))
print('Precision:', '%.3f' % precision_score(y_test, dt2pred))
print('Recall:', '%.3f' % recall_score(y_test, dt2pred))
print('F1 Score:', '%.3f' % f1_score(y_test, dt2pred))
print('AUC score:', '%3.f' % roc_auc_score(y_test, dt2pred))

**=================================================================================================================**

## Cross-Validation

When performing supervised machine learning analysis, it is common to withhold a portion of the data to test the final model's performance. This model testing is performed on the 'unseen' data, which the model was not trained on. This withholding of a portion of the dataset for testing is called Cross-Validation. Cross-Validation can also be used to select hyper-parameters and test the final model. In this section, we will focus on the test data only.

Cross-Validation also helps avoid over-fitting; a complex model could repeat the labels of the samples that it has just seen and, therefore, would have a perfect score but would fail to predict anything useful on the 'unseen' data. Furthermore, a complex model could just be modeling noise.

Cross validation method involves dividing the dataset into 3 parts:

*   training set - is a portion of the data used for training the model
*   validation set - is a portion of the data used to optimize the hyper-parameters of the model. This will     be illustrated in the next lab
*   test set - is a portion of the data used to evaluate if the model generalizes enough to work on the     
    data it was not trained on   
    
`Scikit Learn` library contains many methods that can perform the splitting of the data into training, testing and validation sets. The most popular methods that we will cover in this Jupyter Notebook are:

*   train_test_split - creates a single split into train and test sets
*   K-fold - creates number of k-fold splits, allowing cross validation
*   cross_val_score - evaluates model's score through cross validation

[`cross_val_predict`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML240ENSkillsNetwork783-2023-01-01) is a function that does K-fold cross validation for us, appropriately fitting and transforming at every step of the way.

Note that `cross_val_predict` doesn't use the same model for all steps; the predictions for each row are made when that row is in the validation set. We really have the collected results of 3 (i.e. `kf.num_splits`) different models. 

When we are done, `estimator` is still not fitted. If we want to predict on _new_ data, we still have to train our `estimator`. 


<img src = "cross_validation_diagram.png">
<img src = "k-fold.png">

### K-Fold Validation

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
cv = cross_validate(estimator=dt2, X=X_train, y=y_train, scoring="roc_auc", cv=kf, n_jobs=2, return_train_score=True)
cv

In [ ]:
cv["train_roc_auc"].mean()

In [ ]:
cv["test_roc_auc"].mean()

### Cross Validation Score (Estimator can be any model)

In [ ]:
dtcv_pred = cross_val_score(estimator=dt2, X=X_train, y=y_train, scoring="roc_auc", cv=kf, n_jobs=2)

In [ ]:
dtcv_pred[0:5]

In [ ]:
dtcv_pred.mean()

**=================================================================================================================**

## Extra Tree Classifier ( Note: Future Expansion )

In [ ]:
et1 = ExtraTreeClassifier(random_state=0)

In [ ]:
et1.fit(X_train, y_train)

In [ ]:
et1pred = et1.predict(X_test)

In [ ]:
et1pred[0:5]

In [ ]:
print("Extra Tree Classifier")
print('Accuracy:', '%.3f' % accuracy_score(y_test, et1pred))
print('Precision:', '%.3f' % precision_score(y_test, et1pred))
print('Recall:', '%.3f' % recall_score(y_test, et1pred))
print('F1 Score:', '%.3f' % f1_score(y_test, et1pred))
print('AUC score:', '%3.f' % roc_auc_score(y_test, et1pred))

### Using RandomSearchCV

Randomised grid search is very useful in finding near-optimal hyper parameters for any machine learning models.

Rules of thumb: with 60 iterations, 95% of the time, best 5% sets of parameters can be found, regardless of grid size.

In [ ]:
et = ExtraTreeClassifier(random_state=0)

In [ ]:
parameters = { 'criterion': ['gini', 'entropy'],
               'splitter' : ['random','best'],
               'max_depth': np.arange(2,11,2),
               'min_samples_split': np.arange(2,21,3),
               'min_samples_leaf':np.arange(1,11,2)
             }

In [ ]:
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

In [ ]:
et_randm = RandomizedSearchCV(estimator=et, param_distributions = parameters, cv = 5, n_iter = 55, 
                           n_jobs=2, scoring=scoring, refit='roc_auc', return_train_score=True)

In [ ]:
%%time
et_randm.fit(X_random_train, y_random_train)

In [ ]:
et_randm.best_estimator_

In [ ]:
et_randm.best_score_

In [ ]:
et_randm.best_params_

**=================================================================================================================**

# Bagging and Random Forest Models

Using **ensemble methods** can greatly improve the results achieved with weak machine learning algorithms, also called **weak learners**. Ensemble methods achieve better performance by aggregating the results of many statistically independent models. This process averages out the errors and produces a final better prediction. 

In this lab you will work with a widely used ensemble method known as **bootstrap aggregating** or simply **bagging**. Bagging follows a simple procedure:
1. N learners (machine learning models) are defined. 
2. N subsamples of the training data are created by **Bernoulli sampling with replacement**.
3. The N learners are trained on the subsamples of the training data.
4. The ensemble is scored by averaging, or taking a majority vote, of the predictions from the N learners.

**Classification and regression tree models** are most typically used with bagging methods. The most common such algorithm is know as the **random forest**. The random forest method is highly scalable and generally produces good results, even for complex problems. 

Classification and regression trees tend to be robust to noise or outliers in the training data. This is true for the random forest algorithm as well. 

### Using RandomSearchCV

In [ ]:
# Instantiate the classifier
rf = RandomForestClassifier(random_state=0)

In [ ]:
parameters = {'n_estimators':np.arange(50,200,50),
              'criterion': ('gini','entropy'),
              'max_depth': np.arange(2,11,2),
              'min_samples_split': np.arange(2,11,2),
              'min_samples_leaf': np.arange(1,11,2)
             }

In [ ]:
# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

In [ ]:
rf_randm = RandomizedSearchCV(estimator=rf, param_distributions = parameters, cv = 5, n_iter = 55, 
                           n_jobs=2, scoring=scoring, refit='roc_auc')

In [ ]:
%%time
rf_randm.fit(X_random_train, y_random_train)

In [ ]:
rf_randm.best_estimator_

In [ ]:
rf_randm.best_score_

In [ ]:
rf_randm.best_params_

In [ ]:
# we also find the data for all models evaluated

results = pd.DataFrame(rf_randm.cv_results_)

print(results.shape)

results.head()

In [ ]:
# we can order the different models based on their performance
results.sort_values(by='mean_test_roc_auc', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[["param_n_estimators", "param_min_samples_split","param_max_depth", "param_min_samples_leaf", 
         "param_criterion", "mean_test_accuracy", "mean_test_recall",
         "mean_test_precision","mean_test_f1","mean_test_roc_auc"]].head()

In [ ]:
def make_results(model_name, model_object):
    '''
    Accepts as arguments a model name (your choice - string) and
    a fit GridSearchCV model object.
  
    Returns a pandas df with the F1, recall, precision, and accuracy scores
    for the model with the best mean F1 score across all validation folds.  
    '''

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(mean f1 score)
    best_estimator_results = cv_results.iloc[cv_results['mean_test_roc_auc'].idxmax(), :]

    # Extract accuracy, precision, recall, and f1 score from that row
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy
    rocauc = best_estimator_results.mean_test_roc_auc
  
    # Create table of results
    table = pd.DataFrame()
    table = table.append({'Model': model_name,
                        'F1': f1,
                        'Recall': recall,
                        'Precision': precision,
                        'Accuracy': accuracy,
                        'ROC-AUC' : rocauc  
                        },
                        ignore_index=True
                       )
  
    return table

In [ ]:
# Call the function on our model
rf_result_table = make_results("Random Forest RCV", rf_randm)

rf_result_table

### Tuned Random Forest Model

In [ ]:
rf2 = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=10, min_samples_split=8,
                            min_samples_leaf=5, random_state=0)

In [ ]:
rf2.fit(X_train,y_train)

In [ ]:
rf2_pred = rf2.predict(X_test)

In [ ]:
rf2_pred

### K-Fold Validation

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
cv = cross_validate(estimator=rf2, X=X_train, y=y_train, scoring="roc_auc", cv=kf, n_jobs=2, return_train_score=True)
cv

In [ ]:
cv["train_roc_auc"].mean()

In [ ]:
cv["test_roc_auc"].mean()

### RF Model Evaluation

In [ ]:
rfcm = confusion_matrix(y_test,rf2_pred)
rfcm

In [ ]:
print(classification_report(y_test,rf2_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ConfusionMatrixDisplay.from_estimator(estimator=rf2, X=X_test, y=y_test, ax=ax, display_labels=["No","Yes"])
ax.set_title('Confusion matrix of the classifier', size=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

RocCurveDisplay.from_estimator(estimator=rf2, X=X_test, y=y_test, ax=ax)
ax.set_title('ROC Curve of the classifier', size=15)

plt.show()

### Build a feature importance graph

In [ ]:
importances = rf2.feature_importances_

feature_importances = pd.Series(importances, index=X.columns)

fig, ax = plt.subplots()
feature_importances.plot.bar(ax=ax, figsize=(10,5))

ax.set_title("Random Forest Feature Importances")
ax.tick_params('x', rotation=30)

fig.show()

In [ ]:
feature_importances_df = pd.DataFrame(feature_importances, columns=["importances"])
feature_importances_df = feature_importances_df.sort_values(by='importances')
feature_importances_df

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

sns.barplot(data=feature_importances_df, x=feature_importances_df.importances, y=feature_importances_df.index, orient='h')

ax.set_title("Decision Tree: Feature Importances for Employee Leaving", fontsize=15)

ax.set_xlabel("Importance")
ax.set_ylabel("Feature")

plt.show()

**=================================================================================================================**

## Gradient Boosting (Future Expansion)

### Using RandomSearchCV

In [ ]:
# Instantiate the classifier
gbc = GradientBoostingClassifier(random_state=0)

In [ ]:
parameters = {'loss' : ('log_loss', 'deviance', 'exponential'),
              'learning_rate': np.arange(0.0,1.2,0.2),
              'n_estimators': np.arange(50,250,50),
              'criterion': ('friedman_mse', 'squared_error'),
              'max_depth': np.arange(2,11,2),
              'min_samples_leaf': np.arange(5,25,5),
              'min_samples_split': np.arange(2,25,5)
             }

In [ ]:
# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

In [ ]:
gbc_randm = RandomizedSearchCV(estimator=gbc, param_distributions = parameters, cv = 5, n_iter = 55, 
                           n_jobs=-1, scoring=scoring, refit='roc_auc')

In [ ]:
%%time
gbc_randm.fit(X_random_train, y_random_train)

In [ ]:
gbc_randm.best_estimator_

In [ ]:
gbc_randm.best_score_

In [ ]:
gbc_randm.best_params_

In [ ]:
# we also find the data for all models evaluated

results = pd.DataFrame(gbc_randm.cv_results_)

print(results.shape)

results.head()

In [ ]:
# we can order the different models based on their performance
results.sort_values(by='mean_test_roc_auc', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[["param_n_estimators", "param_min_samples_split","param_max_depth", "param_min_samples_leaf", 
         "param_criterion", "mean_test_accuracy", "mean_test_recall",
         "mean_test_precision","mean_test_f1","mean_test_roc_auc"]].head()

In [ ]:
def make_results(model_name, model_object):
    '''
    Accepts as arguments a model name (your choice - string) and
    a fit GridSearchCV model object.
  
    Returns a pandas df with the F1, recall, precision, and accuracy scores
    for the model with the best mean F1 score across all validation folds.  
    '''

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(mean f1 score)
    best_estimator_results = cv_results.iloc[cv_results['mean_test_roc_auc'].idxmax(), :]

    # Extract accuracy, precision, recall, and f1 score from that row
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy
    rocauc = best_estimator_results.mean_test_roc_auc
  
    # Create table of results
    table = pd.DataFrame()
    table = table.append({'Model': model_name,
                        'F1': f1,
                        'Recall': recall,
                        'Precision': precision,
                        'Accuracy': accuracy,
                        'ROC-AUC' : rocauc  
                        },
                        ignore_index=True
                       )
  
    return table

In [ ]:
# Call the function on our model
gbc_result_table = make_results("Gradient Boosting RCV", gbc_randm)

In [ ]:
gbc_result_table

### Tuned Gradient Boosting Model

In [ ]:
gbc2 = GradientBoostingClassifier(n_estimators=100, criterion='squared_error', max_depth=2, loss='deviance',
                                 learning_rate= 0.8, min_samples_leaf=10, random_state=0)

In [ ]:
gbc2.fit(X_train,y_train)

In [ ]:
gbc2_pred = gbc2.predict(X_test)

In [ ]:
gbc2_pred[0:5]

### K-Fold Validation

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
cv = cross_validate(estimator=gbc2, X=X_train, y=y_train, scoring="roc_auc", cv=kf, n_jobs=2, return_train_score=True)
cv

In [ ]:
cv["train_roc_auc"].mean()

In [ ]:
cv["test_roc_auc"].mean()

### Tuned Gradient Model Evaluation

In [ ]:
gbcm = confusion_matrix(y_test,gbc2_pred)
gbcm

In [ ]:
print(classification_report(y_test, gbc2_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ConfusionMatrixDisplay.from_estimator(estimator=gbc2, X=X_test, y=y_test, ax=ax, display_labels=["No","Yes"])
ax.set_title('Confusion matrix of the classifier', size=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

RocCurveDisplay.from_estimator(estimator=gbc2, X=X_test, y=y_test, ax=ax)
ax.set_title('ROC Curve of the classifier', size=15)

plt.show()

**=================================================================================================================**

## HistGradientBoostingClassifier

### Using RandomSearchCV

In [ ]:
hgbc = HistGradientBoostingClassifier(early_stopping='auto', scoring='roc_auc', random_state=0)

In [ ]:
parameters = {'loss' : ('auto', 'binary_crossentropy', 'categorical_crossentropy'),
              'learning_rate': np.arange(0.0,1.1,0.2),
              'max_depth': np.arange(2,10,2),
              'min_samples_leaf': np.arange(5,21,5),
              'max_depth': np.arange(2,11,1)
             }

In [ ]:
# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

In [ ]:
hgbc_randm = RandomizedSearchCV(estimator=hgbc, param_distributions = parameters, cv = 5, n_iter = 55, 
                           n_jobs=2, scoring=scoring, refit='roc_auc', return_train_score=True)

In [ ]:
%%time
hgbc_randm.fit(X_random_train, y_random_train)

In [ ]:
hgbc_randm.best_estimator_

In [ ]:
hgbc_randm.best_score_

In [ ]:
hgbc_randm.best_params_

In [ ]:
# we also find the data for all models evaluated

results = pd.DataFrame(hgbc_randm.cv_results_)

print(results.shape)

results.head()

In [ ]:
# we can order the different models based on their performance
results.sort_values(by='mean_test_roc_auc', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[["param_loss", "param_max_depth", "param_min_samples_leaf", 
         "param_learning_rate", "mean_test_accuracy", "mean_test_recall",
         "mean_test_precision","mean_test_f1","mean_test_roc_auc"]].head()

In [ ]:
def make_results(model_name, model_object):
    '''
    Accepts as arguments a model name (your choice - string) and
    a fit GridSearchCV model object.
  
    Returns a pandas df with the F1, recall, precision, and accuracy scores
    for the model with the best mean F1 score across all validation folds.  
    '''

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(mean f1 score)
    best_estimator_results = cv_results.iloc[cv_results['mean_test_roc_auc'].idxmax(), :]

    # Extract accuracy, precision, recall, and f1 score from that row
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy
    rocauc = best_estimator_results.mean_test_roc_auc
  
    # Create table of results
    table = pd.DataFrame()
    table = table.append({'Model': model_name,
                        'F1': f1,
                        'Recall': recall,
                        'Precision': precision,
                        'Accuracy': accuracy,
                        'ROC-AUC' : rocauc  
                        },
                        ignore_index=True
                       )
  
    return table

In [ ]:
# Call the function on our model
hist_result_table = make_results("HistGradientBoostingClassifier RCV", hgbc_randm)

hist_result_table

### HistGradientBoosting Model

In [ ]:
hgbc = HistGradientBoostingClassifier(min_samples_leaf=10, max_depth=4, loss='binary_crossentropy',
                                     learning_rate=0.4, early_stopping='auto', scoring='roc_auc', random_state=0)

In [ ]:
hgbc.fit(X_train, y_train)

In [ ]:
hgbc_pred = hgbc.predict(X_test)

In [ ]:
hgbc_pred

### K-Fold Validation

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
cv = cross_validate(estimator=hgbc, X=X_train, y=y_train, scoring="roc_auc", cv=kf, n_jobs=2, return_train_score=True)
cv

In [ ]:
cv["train_roc_auc"].mean()

In [ ]:
cv["test_roc_auc"].mean()

### Tuned Hist Model Evaluation

In [ ]:
hgbcm = confusion_matrix(y_test, hgbc_pred)
hgbcm

In [ ]:
print(classification_report(y_test, hgbc_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ConfusionMatrixDisplay.from_estimator(estimator=hgbcm, X=X_test, y=y_test, ax=ax, display_labels=["No","Yes"])
ax.set_title('Confusion matrix of the classifier', size=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

RocCurveDisplay.from_estimator(estimator=hgbcm, X=X_test, y=y_test, ax=ax)
ax.set_title('ROC Curve of the classifier', size=15)

plt.show()

### Build a feature importance graph

In [ ]:
permimpt = permutation_importance(estimator=hgbc, X=X_train, y=y_train, scoring="roc_auc", n_repeats=5,
                       n_jobs=None, random_state=0)

permimpt

In [ ]:
hgbc_importances = pd.Series(data=permimpt["importances_mean"], index=X.columns)
hgbc_sorted = hgbc_importances.sort_values(ascending=False)

hgbc_sorted

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
sns.barplot(hgbc_sorted.index, hgbc_sorted.values)

ax.set_title("Hist Gradient Boosting Features Importance")
plt.show()

**=================================================================================================================**

# Catboost (Future Expansion)

### Using RandomSearchCV

In [ ]:
cb1 = CatBoostClassifier(random_seed=0)

In [ ]:
parameters = {'learning_rate': np.arange(0.0,1.1,0.2),
              'max_depth': np.arange(2,10,2),
              'min_samples_leaf': np.arange(5,21,5),
              'max_depth': np.arange(2,11,1)
             }

In [ ]:
# Assign a dictionary of scoring metrics to capture
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

In [ ]:
hgbc_randm = RandomizedSearchCV(estimator=hgbc, param_distributions = parameters, cv = 5, n_iter = 55, 
                           n_jobs=2, scoring=scoring, refit='roc_auc', return_train_score=True)

In [ ]:
%%time
hgbc_randm.fit(X_random_train, y_random_train)

In [ ]:
hgbc_randm.best_estimator_

In [ ]:
hgbc_randm.best_score_

In [ ]:
hgbc_randm.best_params_

In [ ]:
# we also find the data for all models evaluated

results = pd.DataFrame(hgbc_randm.cv_results_)

print(results.shape)

results.head()

In [ ]:
# we can order the different models based on their performance
results.sort_values(by='mean_test_roc_auc', ascending=False, inplace=True)

results.reset_index(drop=True, inplace=True)

results[["param_loss", "param_max_depth", "param_min_samples_leaf", 
         "param_learning_rate", "mean_test_accuracy", "mean_test_recall",
         "mean_test_precision","mean_test_f1","mean_test_roc_auc"]].head()

In [ ]:
def make_results(model_name, model_object):
    '''
    Accepts as arguments a model name (your choice - string) and
    a fit GridSearchCV model object.
  
    Returns a pandas df with the F1, recall, precision, and accuracy scores
    for the model with the best mean F1 score across all validation folds.  
    '''

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(mean f1 score)
    best_estimator_results = cv_results.iloc[cv_results['mean_test_roc_auc'].idxmax(), :]

    # Extract accuracy, precision, recall, and f1 score from that row
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy
    rocauc = best_estimator_results.mean_test_roc_auc
  
    # Create table of results
    table = pd.DataFrame()
    table = table.append({'Model': model_name,
                        'F1': f1,
                        'Recall': recall,
                        'Precision': precision,
                        'Accuracy': accuracy,
                        'ROC-AUC' : rocauc  
                        },
                        ignore_index=True
                       )
  
    return table

In [ ]:
# Call the function on our model
hist_result_table = make_results("HistGradientBoostingClassifier RCV", hgbc_randm)

hist_result_table

In [ ]:
df.head()

### Tuned Catboost Model

In [ ]:
cb2 = CatBoostClassifier(random_seed=0, loss_function="Logloss", early_stopping_rounds=100)

In [ ]:
cb2.fit(X_train,y_train, cat_features=["professionaldriver","android"], verbose=False, plot=True);

In [ ]:
cb2.best_score_

In [ ]:
cb2pred = cb2.predict(X_test)

In [ ]:
cb2pred[0:5]

### Tuned Catboost Model Evaluation

In [ ]:
catcm = confusion_matrix(y_test, cb2pred)
catcm

In [ ]:
print(classification_report(y_test, cb2pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ConfusionMatrixDisplay.from_estimator(estimator=cb2, X=X_test, y=y_test, ax=ax, display_labels=["No","Yes"])
ax.set_title('Confusion matrix of the classifier', size=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

RocCurveDisplay.from_estimator(estimator=cb2, X=X_test, y=y_test, ax=ax)
ax.set_title('ROC Curve of the classifier', size=15)

plt.show()

In [ ]:
cb2.feature_importances_

In [ ]:
catboost_importances = pd.Series(data=cb2.feature_importances_, index=X.columns)
catboost_sorted = catboost_importances.sort_values(ascending=False)
catboost_sorted

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
sns.barplot(y = catboost_sorted.index, x = catboost_sorted.values)

ax.set_title("CatBoost Features Importance")
plt.show()

**=================================================================================================================**

## XGBoost (Scikit-Learn)

XGBoost is a widespread implementation of gradient boosting. Let’s discuss some features of XGBoost that make it so attractive.

- XGBoost offers regularization, which allows you to control overfitting by introducing L1/L2 penalties on the weights and biases of each tree. This feature is not available in many other implementations of gradient boosting.
- Another feature of XGBoost is its ability to handle sparse data sets using the weighted quantile sketch algorithm. This algorithm allows us to deal with non-zero entries in the feature matrix while retaining the same computational complexity as other algorithms like stochastic gradient descent.
- XGBoost also has a block structure for parallel learning. It makes it easy to scale up on multicore machines or clusters. It also uses cache awareness, which helps reduce memory usage when training models with large datasets.
- Finally, XGBoost offers out-of-core computing capabilities using disk-based data structures instead of in-memory ones during the computation phase.


<img src = "treesalgo.jpg">

### Cross-validated hyperparameter tuning

The cross-validation process is the same as it was for the decision tree and random forest models. The only difference is that we're tuning different hyperparameters now. The steps are included below as a review. 

For details on cross-validating with `GridSearchCV`, refer back to the [decision tree notebook](https://colab.sandbox.google.com/drive/164Aa1ODOMSIY_5-ZP1PA5afGegTqqjcv?resourcekey=0-hZwiQ1rxwUAol5kaj7-o4w#tuning), or to the [GridSearchCV documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) in scikit-learn.

1. Instantiate the classifier (and set the `random_state`). Note here that we've included a parameter called `objective` whose value is `binary:logistic`. This means that the model is performing a binary classification task that outputs a logistic probability. The objective would be different for different kinds of problems—for instance, if you were trying to predict more than two classes or performing a linear regression on continuous data. Refer to the [XGBoost documentation](https://xgboost.readthedocs.io/en/stable/parameter.html) for more information.

2. Create a dictionary of hyperparameters to search over.

3. Create a dictionary of scoring metrics to capture. 

4. Instantiate the `GridSearchCV` object. Pass as arguments:
  - The classifier (`xgb`)
  - The dictionary of hyperparameters to search over (`cv_params`)
  - The dictionary of scoring metrics (`scoring`)
  - The number of cross-validation folds you want (`cv=5`)
  - The scoring metric that you want GridSearch to use when it selects the "best" model (i.e., the model that performs best on average over all validation folds) (`refit='f1'`)

5. Fit the data (`X_train`, `y_train`) to the `GridSearchCV` object (`xgb_cv`)

Note that we use the `%%time` magic at the top of the cell where we fit the model. This outputs the final runtime of the cell. 

In [ ]:
xgbc1 = XGBClassifier(random_state=0, n_estimators=100, objective='binary:logistic')

In [ ]:
parameters = {'max_depth': np.arange(2,10,2),
              'learning_rate': np.arange(0.1,0.5,0.1),
              'n_estimators':np.arange(50,350,50),
              'min_child_weight': [1,2,3,4,5]
             }

In [ ]:
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

In [ ]:
xgbgs = GridSearchCV(estimator=xgbc,param_grid=parameters,scoring=scoring,
                     n_jobs=2, cv=5, verbose=1, refit='roc_auc', return_train_score=True)

In [ ]:
%%time
xgbgs.fit(X_random_train,y_random_train)

In [ ]:
xgbgs.best_estimator_

In [ ]:
xgbgs.best_score_

In [ ]:
xgbgs.best_params_

In [ ]:
def make_results(model_name, model_object):
    '''
    Accepts as arguments a model name (your choice - string) and
    a fit GridSearchCV model object.
  
    Returns a pandas df with the F1, recall, precision, and accuracy scores
    for the model with the best mean F1 score across all validation folds.  
    '''

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(mean f1 score)
    best_estimator_results = cv_results.iloc[cv_results['mean_test_roc_auc'].idxmax(), :]

    # Extract accuracy, precision, recall, and f1 score from that row
    f1 = best_estimator_results.mean_test_f1
    recall = best_estimator_results.mean_test_recall
    precision = best_estimator_results.mean_test_precision
    accuracy = best_estimator_results.mean_test_accuracy
    rocauc = best_estimator_results.mean_test_roc_auc
  
    # Create table of results
    table = pd.DataFrame()
    table = table.append({'Model': model_name,
                        'F1': f1,
                        'Recall': recall,
                        'Precision': precision,
                        'Accuracy': accuracy,
                        'ROC-AUC' : rocauc  
                        },
                        ignore_index=True
                       )
  
    return table

In [ ]:
# Create xgb model results table
xgb_cv_results = make_results('XGBoost GSCV', xgbgs)
xgb_cv_results

### Using RandomSearchCV

In [ ]:
xgbc = XGBClassifier(random_state=0, n_estimators=100, objective='binary:logistic')

In [ ]:
#xgbc = XGBClassifier(random_state=0, n_estimators=100, objective='softmax:multi')

In [ ]:
parameters = {'max_depth': np.arange(2,11,1),
              'eta': [0.01, 0.1, 0.5, 1.0],
              'n_estimators': np.arange(50,300,50),
              'min_child_weight': np.arange(1,4,1),
              'gamma': np.arange(0,11,2),
              'subsample': np.arange(0.1, 0.9, 0.1),
              'colsample_bytree': np.arange(0.5,0.9,0.1),
              'reg_alpha': np.arange(0.0 , 1.0, 0.2),
              'reg_lambda': np.arange(0.0, 1.0, 0.2)
             }

In [ ]:
scoring = {'accuracy', 'precision', 'recall', 'f1', 'roc_auc'}

In [ ]:
xgbrandm = RandomizedSearchCV(estimator=xgbc, param_distributions = parameters, cv = 5, n_iter = 55, 
                           n_jobs=2, scoring=scoring, refit='roc_auc')

In [ ]:
xgbrandm.fit(X_random_train, y_random_train)

In [ ]:
xgbrandm.best_estimator_

In [ ]:
xgbrandm.best_score_

In [ ]:
xgbrandm.best_params_

### Tuned XGBoost Model

In [ ]:
xgbc2 = XGBClassifier(n_estimators=100, objective='binary:logistic', random_state=0, 
                     colsample_bytree=0.5, eta=0.01, gamma=6, min_child_weight=2, max_depth=7)

In [ ]:
#xgbmodel = XGBClassifier(random_state=0, n_estimators=100, objective='softmax:multi')

In [ ]:
#xgbmodel.fit(X_train,y_train,eval_set=[(X_test,y_test)],eval_metric='rmse',early_stopping_rounds=10)

In [ ]:
xgbc2.fit(X_train, y_train, eval_set=[(X_test,y_test)], eval_metric='error', early_stopping_rounds=30)

In [ ]:
#xgbmodel.fit(X_train_scaled,y_train,eval_set=[(X_test_scaled,y_test)],eval_metric='error',early_stopping_rounds=10)

In [ ]:
#xgbmodel.fit(X_train_scaled,y_train,eval_set=[(X_test_scaled,y_test)],eval_metric='mlogloss',early_stopping_rounds=10)

In [ ]:
xgbc2_pred = xgbc2.predict(X_test)

In [ ]:
xgbc2_pred

### Model Evaluation

In [ ]:
xgbcm = confusion_matrix( y_test, xgbc2_pred)
xgbcm

In [ ]:
print(classification_report(y_test,xgbc2_pred))

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

ConfusionMatrixDisplay.from_estimator(estimator=xgbc2, X=X_test, y=y_test, ax=ax, display_labels=["No","Yes"])
ax.set_title('Confusion matrix of the classifier', size=15)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

RocCurveDisplay.from_estimator(estimator=xgbc2, X=X_test, y=y_test, ax=ax)
ax.set_title('ROC Curve of the classifier', size=15)

plt.show()

### XGBoost Feature importance

The XGBoost library has a function called `plot_importance`, which we imported at the beginning of this notebook. This let's us check the features selected by the model as the most predictive. We can create a plot by calling this function and passing to it the best estimator from our grid search.

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
plot_importance(xgbrandm.best_estimator_, ax=ax);

In [ ]:
xgbmodel.feature_importances_

In [ ]:
feat_importances = pd.Series(xgbmodel.feature_importances_, index=X.columns)

In [ ]:
feat_importances

In [ ]:
feat_importances.nlargest(10).plot(kind='barh', figsize=(10,8))
plt.title('Feature Importances')
plt.show()

### The permutation based importance

In [ ]:
perm_importance = permutation_importance(rf,X_test,y_test, random_state=0, scoring='neg_mean_squared_error')

In [ ]:
sorted_idx = perm_importance.importances_mean.argsort()
plt.figure(figsize=(10,10))
plt.title("Permutation-based Importance")
plt.barh(X.columns[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")
plt.show()

### Compute Importance from SHAP Values

In [ ]:
explainer = shap.TreeExplainer(rf)

In [ ]:
shap_values = explainer.shap_values(X_test)

In [ ]:
shap.summary_plot(shap_values, X_test, plot_type="bar")

In [ ]:
shap.summary_plot(shap_values, X_test)

## XGBoost (Native Booster)

In [ ]:
dtrain = xgb.DMatrix(data=X_train,label=y_train)
dtest = xgb.DMatrix(data=X_test,label=y_test)

In [ ]:
params = {'n_estimators':,
          'learning_rate':,
          'max_depth':,
          'objective': '',
          'num_class': ,
          'seed': 0,
          'eval_metric':''}

In [ ]:
xgbmodel = xgb.train(params=params,dtrain=dtrain,num_boost_round=100,evals=[(dtest,"Test")],
                     early_stopping_rounds=10)

In [ ]:
y_pred = xgbmodel.predict(dtest)

In [ ]:
y_pred

### Cross-Validation (API)

In [ ]:
cv = xgb.cv(params=params,
            dtrain=dtrain,
            num_boost_round=100,
            nfold=5,
            stratified=False,
            folds=None,
            metrics=('merror'),
            obj=None,
            feval=None,
            maximize=False,
            early_stopping_rounds=10,
            fpreproc=None,
            as_pandas=True,
            verbose_eval=None,
            show_stdv=True,
            seed=0,
            callbacks=None,
            shuffle=True,)

In [ ]:
cv

In [ ]:
cv['test-merror-mean'].min()

### Model Evaluation (Classification)

In [ ]:
cm = confusion_matrix(y_test,y_pred)
cm

In [ ]:
fig , ax = plt.subplots(figsize=(10,5))
sns.heatmap(cm, annot=True,fmt='.4g',linewidths=2, cmap='viridis')
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
fig , ax = plt.subplots(figsize=(10,5))
plot_confusion_matrix(xgbmodel,X_test_scaled,y_test,values_format='.4g',ax=ax)
plt.show()

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
plot_roc_curve(xgbmodel,X_test,y_test)
plt.show()

### Table Formatted View

In [ ]:
table = X_test.copy()

In [ ]:
table["True Value"] = y_test.copy()

In [ ]:
table["Predicted"] = np.round(lr_pred,2)

In [ ]:
table

### Model Prediction

In [ ]:
testdata = pd.read_csv()

In [ ]:
answer = xgbmodel.predict(testdata)

In [ ]:
answer

## Compare models

Create a table of results to compare model performance.

In [ ]:
# Create a table of results to compare model performance.

### YOUR CODE HERE ###

table = pd.DataFrame()
table = table.append({'Model': "Tuned Decision Tree",
                        'F1':  0.945422,
                        'Recall': 0.935863,
                        'Precision': 0.955197,
                        'Accuracy': 0.940864
                      },
                        ignore_index=True
                    )

table = table.append({'Model': "Tuned Random Forest",
                        'F1':  0.947306,
                        'Recall': 0.944501,
                        'Precision': 0.950128,
                        'Accuracy': 0.942450
                      },
                        ignore_index=True
                    )

table = table.append({'Model': "Tuned XGBoost",
                        'F1':  f1_score,
                        'Recall': rc_score,
                        'Precision': pc_score,
                        'Accuracy': ac_score
                      },
                        ignore_index=True
                    )

table

**=================================================================================================================**

# Feature selection

Feature selection is the process of choosing features to be used for modeling. In practice, feature selection takes place at multiple points in the PACE process. Although sometimes you will be given a dataset and a defined target variable, most often in practice you will begin with only a question or a problem that you are tasked with solving. In these cases, if you decide that the problem requires a model, you'll then have to:

* Consider what data is available to you
* Decide on what kind of model you need
* Decide on a target variable
* Assemble a collection of features that you think might help predict on your chosen target

This would all take place during the **Plan** phase. 

Then, during the **Analyze** phase, you would perform EDA on the data and reevaluate your variables for appropriateness. For example, can your model handle null values? If not, what do you do with features with a lot of nulls? Perhaps you drop them. This too is feature selection.

But it doesn't end there. Feature selection also occurs during the **Construct** phase. This usually involves building a model, examining which features are most predictive, and then removing the unpredictive features.

There's a lot of work involved in feature selection. In our case, we already have a dataset, and we're not performing thorough EDA on it. But we can still examine the data to ensure that all the features can reasonably be expected to have predictive potential. 

# Filter Methods (Basics)

### Variance Threshold (Numeric Only)

Remember we should apply the variance filter only on numerical variables.

Default Value of Threshold is 0

    If Variance Threshold = 0 (Remove Constant Features )
    If Variance Threshold > 0 (Remove Quasi-Constant Features )


In [ ]:
# threshold_n=0.95

# vt = VarianceThreshold(threshold=(threshold_n* (1 - threshold_n) ))

In [ ]:
#vtfit = vt.fit(X,y)

In [ ]:
#vtfit.variances_

In [ ]:
#vt.get_support()

In [ ]:
#vt.get_feature_names_out()

### Constant and Quasi-constant features with Feature-engine

In this notebook, we will remove constant and quasi-constant features utilizing the new functionality from Feature-engine.

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
X = df.iloc[:,0:17]
y = df.iloc[:,17]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
#X_train

In [ ]:
X_train.shape, X_test.shape

### Remove constant features

The DropConstantFeatures class from Feature-engine finds and removes constant and quasi-constant features from a dataset. We can remove constant features by setting the parameter tol to 1, or quasi-constant with smaller values for tol.

In [ ]:
sel = DropConstantFeatures(tol=1, variables=None, missing_values='raise')

In [ ]:
sel.fit(X_train)

In [ ]:
# list of constant features

sel.features_to_drop_

In [ ]:
# remove constant features from the data

X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

X_train.shape, X_test.shape

### Remove quasi-constant features

In [ ]:
sel = DropConstantFeatures(tol=0.90, variables=None, missing_values='raise') #90% majority observations

In [ ]:
sel.fit(X_train)

In [ ]:
# list of quasi-constant features

sel.features_to_drop_

In [ ]:
# percentage of observations showing each of the different values
# of the variable

var = sel.features_to_drop_[0]

X_train[var].value_counts(normalize=True)

In [ ]:
#remove the quasi-constant features

X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

In [ ]:
X_train.shape, X_test.shape

### Duplicated features with Feature-engine

In this notebook, we will identify and remove duplicated features with Feature-engine.

In [ ]:
# set up the selector
sel = DropDuplicateFeatures(variables=None, missing_values='raise')

In [ ]:
# find the duplicate features, this might take a while
sel.fit(X_train)

In [ ]:
# these are the pairs of duplicated features
# each set are duplicates

sel.duplicated_feature_sets_

In [ ]:
# these are the features that will be dropped
# 1 from each of the pairs above

sel.features_to_drop_

In [ ]:
# remove the duplicated features

X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

In [ ]:
X_train.shape, X_test.shape

# Filter Methods (Correlation)

Correlation Feature Selection evaluates subsets of features on the basis of the following hypothesis: "Good feature subsets contain features highly correlated with the target, yet uncorrelated to each other".

### Correlation with Feature-engine

- The DropCorrelatedFeatures class from Feature-engine does a similar job to the brute force approach that we described earlier.

- The SmartCorrelationSelection allows us to select a feature from each correlated group based on model performance, number of missing values, cardinality or variance.

In [ ]:
# set up the selector

sel = DropCorrelatedFeatures(
    threshold=0.8,
    method='pearson',
    missing_values='ignore'
)

In [ ]:
# find correlated features

sel.fit(X_train)

In [ ]:
# each set contains a group of correlated features

sel.correlated_feature_sets_

In [ ]:
# drop correlated features

X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

In [ ]:
X_train.shape, X_test.shape

## SmartCorrelationSelection

### Model Performance

We will keep a feature from each correlation group based on the performance of a model.

In [ ]:
lr = LinearRegression()

In [ ]:
# correlation selector
sel = SmartCorrelatedSelection(
    variables=None, # if none, selector examines all numerical variables
    method="pearson",
    threshold=0.8,
    missing_values="raise",
    selection_method="model_performance",
    estimator=lr,
    scoring="neg_root_mean_squared_error",
    cv=5
)

In [ ]:
# this may take a while, because we are training
# a random forest per correlation group

sel.fit(X_train, y_train)

In [ ]:
# groups of correlated features

sel.correlated_feature_sets_

In [ ]:
X_train.shape, X_test.shape

**In this group, several features are highly correlated. Which one should we keep and which ones should we remove?**

One criteria to select which features to use from this group, would be to use those with **less missing data**. 

Our dataset contains no missing values, so this is not an option. But keep this in mind when you work with your own datasets.

**Note**

None of the 2 procedures for removing correlated features are perfect, and some correlated features may escape the loops of code. So it might be worthwhile checking that after removing the correlated features, there are no correlated features left in the dataset. If there are, repeat the procedure to remove the remaining ones.

**=================================================================================================================**

# Filter Methods (Statistical Tests)

## Mutual information

The mutual information measures the reduction in uncertainty in variable A when variable B is known. 

To select variables, we are interested in the mutual information between the predictor variables and the target. Higher mutual information values, indicate little uncertainty about the target Y given the predictor X.


In [ ]:
# determine the mutual information
mi = mutual_info_classif(X_train, y_train)

# and make a bar  plot
mi = pd.Series(mi)
mi.index = X_train.columns
mi.sort_values(ascending=False).plot.bar(figsize=(20,6))
plt.xticks(rotation=45)
plt.ylabel('Mutual Information')
plt.show()

In [ ]:
# here we will select the top 10 features
# based on their mutual information value

# select features
selkbest = SelectKBest(mutual_info_classif, k=5)

In [ ]:
selkbest.fit(X_train, y_train)

In [ ]:
# display features
X_train.columns[selkbest.get_support()]

In [ ]:
# to remove the rest of the features:

X_train = selkbest.transform(X_train)
X_test = selkbest.transform(X_test)

In [ ]:
X_train.shape, X_test.shape    # Can start training ML models

In [ ]:
# Select the features in the top percentile
selpercent = SelectPercentile(mutual_info_classif, percentile=30) # Based on no of features to decide

In [ ]:
selpercent.fit(X_train, y_train)

In [ ]:
# display the features
X_train.columns[selpercent.get_support()]

In [ ]:
# to remove the rest of the features:

X_train = selpercent.transform(X_train)
X_test = selpercent.transform(X_test)

## Univariate feature selection

Univariate feature selection works by selecting the best features based on univariate statistical tests (ANOVA). The methods estimate the degree of linear dependency between two random variables. In this case, any of the predictor variables and the target. 

ANOVA assumes a linear relationship between the feature and the target and that the variables follow a Gaussian distribution. If this is not true, the result of this test may not be useful.

These may not always be the case for the variables in your dataset, so if looking to implement these procedure, you will need to corroborate these assumptions.

In [ ]:
# univariate anova
univariate = f_classif(X_train, y_train)

# plot values
univariate = pd.Series(univariate[1])
univariate.index = X_train.columns
univariate.sort_values(ascending=False).plot.bar(figsize=(20,6))
plt.show()

In [ ]:
# select the top 10 features
selkbest = SelectKBest(f_classif, k=5)

In [ ]:
selkbest.fit(X_train, y_train)

In [ ]:
# display selected feature names
X_train.columns[selkbest.get_support()]

In [ ]:
# to remove the rest of the features:

X_train = selpercent.transform(X_train)
X_test = selpercent.transform(X_test)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
# select features in top 10th percentile
selpercent= SelectPercentile(f_classif, percentile=30)

In [ ]:
selpercent.fit(X_train, y_train)

In [ ]:
# display selected feature names
X_train.columns[selpercent.get_support()]

In [ ]:
# to remove the rest of the features:

X_train = selpercent.transform(X_train)
X_test = selpercent.transform(X_test)

In [ ]:
X_train.shape, X_test.shape

**=================================================================================================================**

# Filter Methods (Other Methods)

## Univariate Performance with Feature-engine

This procedure works as follows:

- Train a ML model per every single feature
- Determine the performance of the models
- Select features if model performance is above a certain threshold

The C value in Logistic Regression is an user adjustable parameter that controls regularisation. In simple terms, higher values of C will instruct our model to fit the training set as best as possible, while lower C values will favour a simple models with coefficients closer to zero.


In [ ]:
# set up the machine learning model
lr = LogisticRegression(penalty='l2', C=1000, random_state=0, solver='lbfgs', max_iter=1000)

In [ ]:
# set up the selector
sel = SelectBySingleFeaturePerformance(
    variables=None,
    estimator=lr,
    scoring="roc_auc",
    cv=5,
    threshold=0.5
)

In [ ]:
# find predictive features
sel.fit(X_train, y_train)

In [ ]:
# the transformer stores a dictionary of feature:metric pairs
# notice that the roc can be positive or negative.
# the selector selects based on the absolute value

#In general, an AUC of 0.5 suggests no discrimination 
#(i.e., ability to diagnose patients with and without the disease or condition based on the test), 
#0.7 to 0.8 is considered acceptable, 0.8 to 0.9 is considered excellent, and more than 0.9 is considered outstanding

sel.feature_performance_

In [ ]:
pd.Series(sel.feature_performance_).sort_values(ascending=False).plot.bar(figsize=(20, 5))
plt.title('Performance of ML models trained with individual features', size=15)
plt.xticks(rotation=45)
plt.ylabel('ROC Score')
plt.show()

In [ ]:
# same plot but taking the absolute value of the r2

# np.abs(pd.Series(sel.feature_performance_)).sort_values(ascending=False).plot.bar(figsize=(20, 5))
# plt.title('Performance of ML models trained with individual features', size=15)
# plt.ylabel('ROC Score')
# plt.show()

In [ ]:
# the features that will be removed

sel.features_to_drop_

In [ ]:
# select features in the dataframes

X_train = sel.transform(X_train)
X_test = sel.transform(X_test)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train.columns

**=================================================================================================================**

# Step forward feature selection

Step forward feature selection starts by training a machine learning model for each feature in the dataset and selecting, as the starting feature, the one that returns the best performing model according to the evaluation criteria we choose.

In the second step, it creates machine learning models for all combinations of the feature selected in the previous step and a second feature. It selects the pair that produces the best performing algorithm.

It continues by adding 1 feature at a time to the features that were selected in previous steps, until a stopping criteria is reached.

In theory, models with more features perform better. The algorithm will continue adding new features until a certain criteria is met. For example, until the model performance does not increase beyond a certain threshold. Or, as we show in this notebook, until a certain number of features are selected.

The model performance metric can be the roc_auc for classification and the r squared for regression, for example, and it is determined by the user. 

Step forward feature selection is called a greedy procedure because it evaluates many possible single, double, triple, and so on feature combinations. Therefore, it is very computationally expensive and, sometimes, if the feature space is big enough, even unfeasible.

Scikit-learn provides various stopping criteria to stop the search:

* when a certain number of features is reached (like MLXtend) (arbitrary)
* if the performance does not increase beyond a certain threshold (ideal but expensive)
* selects half of the features (arbitrary)

### Step Forward Feature Selection Regression

In [ ]:
df = pd.read_csv("carpricemod.csv")

In [ ]:
df.shape

In [ ]:
df.head(1)

In [ ]:
X = df.iloc[:,0:14]
y = df.iloc[:,14]

In [ ]:
X.values, y.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# within the SFS we indicate:

# 1) the algorithm we want to create, in this case RandomForests
# (note that I use few trees to speed things up)

# 2) the stopping criteria: see sklearn documentation for more details

# 3) whether to perform step forward or step backward

# 4) the evaluation metric: in this case the roc_auc
# 5) the cross-validation

# this is going to take a while, do not despair

sfs = SFS(estimator=LinearRegression(), 
          n_features_to_select=6,
          direction='forward',
          scoring='r2',
          cv=5,
          n_jobs=-1)

In [ ]:
sfs = sfs.fit(X_train, y_train)

In [ ]:
sfs.get_feature_names_out()

In [ ]:
sfs.n_features_to_select

### Step Forward Feature Selection Classification

In [ ]:
#df = pd.read_csv(".csv")

In [ ]:
df.shape

In [ ]:
X = df.iloc[:,0:8]
y = df.iloc[:,8]

In [ ]:
X.values, y.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# within the SFS we indicate:

# 1) the algorithm we want to create, in this case RandomForests
# (note that I use few trees to speed things up)

# 2) the stopping criteria: see sklearn documentation for more details

# 3) whether to perform step forward or step backward

# 4) the evaluation metric: in this case the roc_auc
# 5) the cross-validation

# this is going to take a while, do not despair

sfs = SFS(estimator=LogisticRegression(random_state=0), 
          n_features_to_select=4,
          direction='forward',
          scoring='f1',
          cv=5,
          n_jobs=-1)

In [ ]:
sfs = sfs.fit(X_train, y_train)

In [ ]:
sfs.get_feature_names_out()

# Step backward feature selection

Step Backward Feature Selection starts by fitting a model using all the features in the data set and determining its performance. 

Then, it trains models on all possible combinations of all features, minus one, and removes the feature that returns the model with the lowest performance.

In the third step, it trains models in all possible combinations of the features remaining from step 2, minus 1 feature, and removes the feature that produced the lowest performing model.

The algorithm stops when a certain criteria determined by the user is met. This criteria could be that the model performance does not decrease beyond a certain threshold, or alternatively, as we show in this notebook, when we reach a certain number of selected features.

The evaluation metric can be the roc_auc for classification or the r squared for regression, for example, and is determined by the user.

Step Backward Feature Selection is called greedy because it evaluates all possible n, and then n-1 and n-2 and so on feature combinations. Therefore, it is very computationally expensive and sometimes, if the feature space is big enough, even unfeasible.

Scikit-learn provides various stopping criteria to stop the search:

* when a certain number of features is reached (like MLXtend) (arbitrary)
* if the performance does not increase beyond a certain threshold (ideal but expensive)
* selects half of the features (arbitrary)

### Step Forward Feature Selection Regression

In [ ]:
df = pd.read_csv(".csv")

In [ ]:
df.shape

In [ ]:
df.head(1)

In [ ]:
X = df.iloc[:,0:14]
y = df.iloc[:,14]

In [ ]:
X.values, y.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# within the SFS we indicate:

# 1) the algorithm we want to create, in this case RandomForests
# (note that I use few trees to speed things up)

# 2) the stopping criteria: see sklearn documentation for more details

# 3) whether to perform step forward or step backward

# 4) the evaluation metric: in this case the roc_auc
# 5) the cross-validation

# this is going to take a while, do not despair

sfs = SFS(estimator=LinearRegression(), 
          n_features_to_select=6,
          direction='backward',
          scoring='r2',
          cv=5,
          n_jobs=-1)

In [ ]:
sfs = sfs.fit(X_train, y_train)

In [ ]:
sfs.get_feature_names_out()

In [ ]:
sfs.n_features_to_select

### Step Backward Feature Selection Classification

In [ ]:
#df = pd.read_csv(".csv")

In [ ]:
df.shape

In [ ]:
X = df.iloc[:,0:8]
y = df.iloc[:,8]

In [ ]:
X.values, y.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# within the SFS we indicate:

# 1) the algorithm we want to create, in this case RandomForests
# (note that I use few trees to speed things up)

# 2) the stopping criteria: see sklearn documentation for more details

# 3) whether to perform step forward or step backward

# 4) the evaluation metric: in this case the roc_auc
# 5) the cross-validation

# this is going to take a while, do not despair

sfs = SFS(estimator=LogisticRegression(random_state=0), 
          n_features_to_select=4,
          direction='backward',
          scoring='f1',
          cv=5,
          n_jobs=-1)

In [ ]:
sfs = sfs.fit(X_train, y_train)

In [ ]:
sfs.get_feature_names_out()

**=================================================================================================================**

**=================================================================================================================**

## Data Pipelines

Data Pipelines simplify the steps of processing the data. We use the module <code>Pipeline</code> to create a pipeline. 
`Pipeline` lets you chain together multiple operators on your data that both have a `fit` method.

### Combine multiple processing steps into a `Pipeline`

A pipeline contains a series of steps, where a step is ("name of step", actual_model). The "name of step" string is only used to help you identify which step you are on, and to allow you to specify parameters at that step.  

In [ ]:
Input = [('scaler', StandardScaler()), ('lr', LinearRegression())]

In [ ]:
Input = [ ('polynomial', PolynomialFeatures(include_bias=False,degree=2)),('ss',StandardScaler() ), ('model',Ridge(alpha=1))]

In [ ]:
estimator = Pipeline(Input, verbose=True)

In [ ]:
estimator.fit(X_train,y_train)

In [ ]:
lr_pred = estimator.predict(X_test)
lr_pred

In [ ]:
estimator.score(X_train, y_train)

In [ ]:
estimator.score(X_test,y_test)

**=================================================================================================================**

# Pickle  

When models take a long time to fit, you don’t want to have to fit them more than once. If your kernel disconnects or you shut down the notebook and lose the cell’s output, you’ll have to refit the model, which can be frustrating and time-consuming. 

`pickle` is a tool that saves the fit model object to a specified location, then quickly reads it back in. It also allows you to use models that were fit somewhere else, without having to train them yourself.

### Save the Model
This step will ***W***rite (i.e., save) the model, in ***B***inary (hence, `wb`), to the folder designated by the above path. In this case, the name of the file we're writing is `rf_cv_model.pickle`.

In [ ]:
filename = 'model.sav'
dump(xgbnew,open(filename,'wb'))

Once we save the model, we'll never have to re-fit it when we run this notebook. Ideally, we could open the notebook, select "Run all," and the cells would run successfully all the way to the end without any model retraining. 

For this to happen, we'll need to return to the cell where we defined our grid search and comment out the line where we fit the model. Otherwise, when we re-run the notebook, it would refit the model. 

Similarly, we'll also need to go back to where we saved the model as a pickle and comment out those lines.  

Next, we'll add a new cell that reads in the saved model from the folder we already specified. For this, we'll use `rb` (read binary) and be sure to assign the model to the same variable name as we used above, `rf_cv`.

### Load the Model

In [ ]:
loaded_model = load(open(filename,'rb'))

In [ ]:
loaded_model

#### Python code done by Dennis Lam